In [0]:
import sys
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as dset
from torchsummary import summary

In [0]:
print("--sys.version--")
print(sys.version)

print("\n--torch.__version__--")
print(torch.__version__)

--sys.version--
3.6.8 (default, Jan 14 2019, 11:02:34) 
[GCC 8.0.1 20180414 (experimental) [trunk revision 259383]]

--torch.__version__--
1.1.0


In [0]:
batch_size = 100
total_epoch = 500
learning_rate = 0.01
use_cuda = torch.cuda.is_available()
criterion = nn.CrossEntropyLoss()

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
train_dataset = dset.ImageFolder(root="/content/gdrive/My Drive/Colab Notebooks/pytorch/gender classification/train", transform=transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]))
test_dataset = dset.ImageFolder(root="/content/gdrive/My Drive/Colab Notebooks/pytorch/gender classification/test", transform=transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]))
test2_dataset = dset.ImageFolder(root="/content/gdrive/My Drive/Colab Notebooks/pytorch/gender classification/test2", transform=transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]))

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)
test2_loader = torch.utils.data.DataLoader(dataset=test2_dataset, batch_size=batch_size, shuffle=False)

In [0]:
def train(model, train_loader):
  model.train()
  
  optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
  losses = []
  for i, (image, label) in enumerate(train_loader):
    
    if use_cuda:
      image = image.cuda()
      label = label.cuda()
      
    pred_label = model(image)
    loss = criterion(pred_label, label)
    losses.append(loss.item())
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
  avg_loss = sum(losses)/len(losses)
  return avg_loss

In [0]:
def eval(model, test_loader):
  model.eval()
  device = next(model.parameters()).device.index
  
  total_cnt = 0
  correct_cnt = 0
  
  for i, (image, label) in enumerate(test_loader):
    if use_cuda:
      image = image.cuda()
      label = label.cuda()
      
      out = model(image)
      _, pred_label = torch.max(out.data, 1)
      total_cnt += image.data.size()[0]
      correct_cnt += (pred_label == label.data).sum().item()
      
    return correct_cnt / total_cnt

In [0]:
class SimpleMLP(nn.Module):
  def __init__(self):
    super(SimpleMLP, self).__init__()
    self.fc1 = nn.Linear(3*32*32, 8*28*28) 
    self.act1 = nn.ReLU()
    self.fc2 = nn.Linear(8*28*28, 8*24*24)
    self.act2 = nn.ReLU()    
    self.fc3 = nn.Linear(8*24*24, 16*8*8)
    self.act3 = nn.ReLU()
    self.fc4 = nn.Linear(16*8*8, 16*4*4)
    self.act4 = nn.ReLU()
    
    # Output layer
    self.out = nn.Linear(16*4*4, 10)
    
  def forward(self, x):
    x = x.view(-1, 3*32*32)
    x = self.act1(self.fc1(x))
    x = self.act2(self.fc2(x))
    x = self.act3(self.fc3(x))
    x = self.act4(self.fc4(x))
    
    out = self.out(x)
    return out

In [0]:
class SimpleMLP_Sigmoid(nn.Module):
  def __init__(self):
    super(SimpleMLP_Sigmoid, self).__init__()
    self.fc1 = nn.Linear(3*32*32, 8*28*28) 
    self.act1 = nn.Sigmoid()
    self.fc2 = nn.Linear(8*28*28, 8*24*24)
    self.act2 = nn.Sigmoid()    
    self.fc3 = nn.Linear(8*24*24, 16*8*8)
    self.act3 = nn.Sigmoid()
    self.fc4 = nn.Linear(16*8*8, 16*4*4)
    self.act4 = nn.Sigmoid()
    
    # Output layer
    self.out = nn.Linear(16*4*4, 10)
    
  def forward(self, x):
    x = x.view(-1, 3*32*32)
    x = self.act1(self.fc1(x))
    x = self.act2(self.fc2(x))
    x = self.act3(self.fc3(x))
    x = self.act4(self.fc4(x))
    
    out = self.out(x)
    return out

In [0]:
class SimpleCNN(nn.Module):
  def __init__(self):
    super(SimpleCNN, self).__init__()
    # Convolution layer
    self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=2, padding=1)
    self.act1 = nn.ReLU()
    self.pool1 = nn.MaxPool2d(kernel_size=2)
    
    self.conv2 = nn.Conv2d(64, 192, kernel_size=3, padding=1)
    self.act2 = nn.ReLU()
    self.pool2 = nn.MaxPool2d(kernel_size=2)
    
    self.conv3 = nn.Conv2d(192, 384, kernel_size=3, padding=1)
    self.act3 = nn.ReLU()
    
    self.conv4 = nn.Conv2d(384, 256, kernel_size=3, padding=1)
    self.act4 = nn.ReLU()
    self.pool3 = nn.MaxPool2d(kernel_size=2)
    
    # Fully-Connected layer
    self.fc1 = nn.Linear(256 * 2 * 2, 1000)
    self.act5 = nn.ReLU()
    self.output = nn.Linear(1000, 10)
    
    
  def forward(self, x):
    x = self.pool1(self.act1(self.conv1(x)))
    x = self.pool2(self.act2(self.conv2(x)))
    x = self.act3(self.conv3(x))
    x = self.act4(self.conv4(x))
    x = self.pool3(x)
    
    x = x.view(-1, 256 * 2 * 2)
    
    x = self.act5(self.fc1(x))
    out = self.output(x)
    return out

In [0]:
class SimpleVGG(nn.Module):
  def __init__(self):
    super(SimpleVGG, self).__init__()
    self.conv1 = nn.Conv2d(3, 64, kernel_size=(3,3), padding=(1,1))
    self.act1 = nn.ReLU()
    self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
    
    
    self.conv2 = nn.Conv2d(64, 128, kernel_size=(3,3), padding=(1,1))
    self.act2 = nn.ReLU()
    self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
    
    self.conv3_1 = nn.Conv2d(128, 256, kernel_size=(3,3), padding=(1,1))
    self.act3_1 = nn.ReLU()
    self.conv3_2 = nn.Conv2d(256, 256, kernel_size=(3,3), padding=(1,1))
    self.act3_2 = nn.ReLU()
    self.conv3_3 = nn.Conv2d(256, 256, kernel_size=(3,3), padding=(1,1))
    self.act3_3 = nn.ReLU()
    self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)
    
    self.conv4_1 = nn.Conv2d(256, 512, kernel_size=(3,3), padding=(1,1))
    self.act4_1 = nn.ReLU()
    self.conv4_2 = nn.Conv2d(512, 512, kernel_size=(3,3), padding=(1,1))
    self.act4_2 = nn.ReLU()
    self.conv4_3 = nn.Conv2d(512, 512, kernel_size=(3,3), padding=(1,1))
    self.act4_3 = nn.ReLU()
    self.pool4 = nn.AvgPool2d(kernel_size=2, stride=2)
    
    # Output layer
    self.fc1 = nn.Linear(512 * 2 * 2, 512)
    self.act5 = nn.ReLU()
    self.out = nn.Linear(512, 10)
    
  def forward(self, x):
    x1 = x
    x2 = self.act1(self.conv1(x1))
    x3 = self.pool1(x2)
    
    x4 = self.act2(self.conv2(x3))
    x5 = self.pool2(x4)
    
    x6 = self.act3_1(self.conv3_1(x5))
    x7 = self.act3_2(self.conv3_2(x6))
    x8 = self.act3_3(self.conv3_3(x7))
    x9 = self.pool3(x8)
    
    x10 = self.act4_1(self.conv4_1(x9))
    x11 = self.act4_2(self.conv4_2(x10))
    x12 = self.act4_3(self.conv4_3(x11))
    x13 = self.pool4(x12)
    
    x14 = x13.view(-1, 512 * 2 * 2)
    
    x15 = self.act5(self.fc1(x14))
    
    out = self.out(x15)
    return out

In [0]:
class SimpleResNet(nn.Module):
  def __init__(self):
    super(SimpleResNet, self).__init__()
    self.conv1 = nn.Conv2d(3, 64, kernel_size=(3,3), padding=(1,1))
    self.act1 = nn.ReLU()
    self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
    
    
    self.conv2 = nn.Conv2d(64, 128, kernel_size=(3,3),padding=(1,1))
    self.act2 = nn.ReLU()
    self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
    
    self.conv3_1 = nn.Conv2d(128, 256, kernel_size=(3,3), padding=(1,1))
    self.act3_1 = nn.ReLU()
    self.conv3_2 = nn.Conv2d(256, 256, kernel_size=(3,3), padding=(1,1))
    self.act3_2 = nn.ReLU()
    self.conv3_3 = nn.Conv2d(256, 256, kernel_size=(3,3), padding=(1,1))
    self.act3_3 = nn.ReLU()
    self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)
    
    self.conv4_1 = nn.Conv2d(256, 512, kernel_size=(3,3), padding=(1,1))
    self.act4_1 = nn.ReLU()
    self.conv4_2 = nn.Conv2d(512, 512, kernel_size=(3,3), padding=(1,1))
    self.act4_2 = nn.ReLU()
    self.conv4_3 = nn.Conv2d(512, 512, kernel_size=(3,3), padding=(1,1))
    self.act4_3 = nn.ReLU()
    self.pool4 = nn.AvgPool2d(kernel_size=2, stride=2)
    
    # Output layer
    self.fc1 = nn.Linear(512 * 2 * 2, 512)
    self.act5 = nn.ReLU()
    self.out = nn.Linear(512, 10)
  
  def forward(self, x):
    x1 = x
    x2 = self.act1(self.conv1(x1))
    x3 = self.pool1(x2)
    
    x4 = self.act2(self.conv2(x3))
    x5 = self.pool2(x4)
    
    x6 = self.act3_1(self.conv3_1(x5))
    x7 = self.act3_2(self.conv3_2(x6))
    x8 = self.act3_3(self.conv3_3(x7) + x6)
    x9 = self.pool3(x8)
    
    x10 = self.act4_1(self.conv4_1(x9))
    x11 = self.act4_1(self.conv4_2(x10))
    x12 = self.act4_1(self.conv4_3(x11) + x10)
    x13 = self.pool4(x12)
    
    x14 = x13.view(-1, 512 * 2 * 2)
    
    x15 = self.act5(self.fc1(x14))
    
    out = self.out(x15)
    return out           

In [0]:
mlp_model = SimpleMLP().cuda()
train_loss_lst = []
test_accuracy_lst = []
test2_accuracy_lst = []
for epoch in range(total_epoch):
  train_loss = train(mlp_model, train_loader)
  train_loss_lst.append(train_loss)
  test_accuracy = eval(mlp_model, test_loader)
  test_accuracy_lst.append(test_accuracy)
  test2_accuracy = eval(mlp_model, test2_loader)
  test2_accuracy_lst.append(test_accuracy2)

  print(epoch+1, "loss :", train_loss)
  print("test_Accuracy :", test_accuracy, "test2_Accuracy :", test2_accuracy)
  
summary(mlp_model, input_size = (3, 32, 32))

1 loss : 2.203436589241028
test_Accuracy : 0.54 test2_Accuracy : 0.51
2 loss : 1.9720926344394685
test_Accuracy : 0.64 test2_Accuracy : 0.6
3 loss : 1.3796346545219422
test_Accuracy : 0.5 test2_Accuracy : 0.5
4 loss : 0.7995555192232132
test_Accuracy : 0.63 test2_Accuracy : 0.6
5 loss : 0.6901302486658096
test_Accuracy : 0.62 test2_Accuracy : 0.57
6 loss : 0.6564530730247498
test_Accuracy : 0.67 test2_Accuracy : 0.6
7 loss : 0.6260393679141998
test_Accuracy : 0.67 test2_Accuracy : 0.62
8 loss : 0.5978148311376572
test_Accuracy : 0.64 test2_Accuracy : 0.59
9 loss : 0.5596233040094376
test_Accuracy : 0.65 test2_Accuracy : 0.6
10 loss : 0.5250559255480767
test_Accuracy : 0.69 test2_Accuracy : 0.65
11 loss : 0.5028508350253105
test_Accuracy : 0.66 test2_Accuracy : 0.6
12 loss : 0.4721181780099869
test_Accuracy : 0.73 test2_Accuracy : 0.58
13 loss : 0.4598778232932091
test_Accuracy : 0.66 test2_Accuracy : 0.63
14 loss : 0.4397169783711433
test_Accuracy : 0.72 test2_Accuracy : 0.63
15 loss :

In [0]:
mlp_model2 = SimpleMLP_Sigmoid().cuda()
train_loss_lst = []
test_accuracy_lst = []
test2_accuracy_lst = []
for epoch in range(total_epoch):
  train_loss = train(mlp_model2, train_loader)
  train_loss_lst.append(train_loss)
  test_accuracy = eval(mlp_model2, test_loader)
  test_accuracy_lst.append(test_accuracy)
  test2_accuracy = eval(mlp_model2, test2_loader)
  test2_accuracy_lst.append(test2_accuracy)
  
  print(epoch+1, "loss :", train_loss)
  print("test_Accuracy :", test_accuracy, "test2_Accuracy :", test2_accuracy)
  
summary(mlp_model2, input_size = (3, 32, 32))

1 loss : 1.298126655817032
test_Accuracy : 0.5 test2_Accuracy : 0.5
2 loss : 0.7892580151557922
test_Accuracy : 0.5 test2_Accuracy : 0.5
3 loss : 0.7420139610767365
test_Accuracy : 0.5 test2_Accuracy : 0.5
4 loss : 0.7271057337522506
test_Accuracy : 0.5 test2_Accuracy : 0.5
5 loss : 0.718601468205452
test_Accuracy : 0.5 test2_Accuracy : 0.5
6 loss : 0.7127487123012543
test_Accuracy : 0.5 test2_Accuracy : 0.5
7 loss : 0.7095115512609482
test_Accuracy : 0.5 test2_Accuracy : 0.5
8 loss : 0.7074590235948562
test_Accuracy : 0.5 test2_Accuracy : 0.5
9 loss : 0.7071517467498779
test_Accuracy : 0.5 test2_Accuracy : 0.5
10 loss : 0.705679976940155
test_Accuracy : 0.5 test2_Accuracy : 0.5
11 loss : 0.7057744920253753
test_Accuracy : 0.5 test2_Accuracy : 0.5
12 loss : 0.7033911615610122
test_Accuracy : 0.5 test2_Accuracy : 0.5
13 loss : 0.7025942087173462
test_Accuracy : 0.5 test2_Accuracy : 0.5
14 loss : 0.7008459329605102
test_Accuracy : 0.5 test2_Accuracy : 0.5
15 loss : 0.7017078220844268
tes

In [0]:
cnn_model = SimpleCNN().cuda()
train_loss_lst = []
test_accuracy_lst = []
test2_accuracy_lst = []
for epoch in range(total_epoch):
  train_loss = train(cnn_model, train_loader)
  train_loss_lst.append(train_loss)
  test_accuracy = eval(cnn_model, test_loader)
  test_accuracy_lst.append(test_accuracy)
  test2_accuracy = eval(cnn_model, test2_loader)
  test2_accuracy_lst.append(test2_accuracy)
  
  print(epoch+1, "loss :", train_loss)
  print("test_Accuracy :", test_accuracy, "test2_Accuracy :", test2_accuracy)
  
summary(cnn_model, input_size = (3,32,32))

1 loss : 2.228582727909088
test_Accuracy : 0.5 test2_Accuracy : 0.5
2 loss : 2.040851390361786
test_Accuracy : 0.5 test2_Accuracy : 0.5
3 loss : 1.3585671931505203
test_Accuracy : 0.5 test2_Accuracy : 0.49
4 loss : 0.7358536511659622
test_Accuracy : 0.5 test2_Accuracy : 0.52
5 loss : 0.7021587818861008
test_Accuracy : 0.5 test2_Accuracy : 0.5
6 loss : 0.6989879429340362
test_Accuracy : 0.5 test2_Accuracy : 0.5
7 loss : 0.701302033662796
test_Accuracy : 0.68 test2_Accuracy : 0.72
8 loss : 0.6969967991113662
test_Accuracy : 0.57 test2_Accuracy : 0.51
9 loss : 0.6870907098054886
test_Accuracy : 0.5 test2_Accuracy : 0.5
10 loss : 0.6902557104825974
test_Accuracy : 0.5 test2_Accuracy : 0.5
11 loss : 0.69667888879776
test_Accuracy : 0.55 test2_Accuracy : 0.57
12 loss : 0.68494793176651
test_Accuracy : 0.5 test2_Accuracy : 0.5
13 loss : 0.6812837928533554
test_Accuracy : 0.52 test2_Accuracy : 0.51
14 loss : 0.6767796784639358
test_Accuracy : 0.5 test2_Accuracy : 0.5
15 loss : 0.69035117924213

In [0]:
vgg_model = SimpleVGG().cuda()
train_loss_lst = []
test_accuracy_lst = []
test2_accuracy_lst = []
for epoch in range(total_epoch):
  train_loss = train(vgg_model, train_loader)
  train_loss_lst.append(train_loss)
  test_accuracy = eval(vgg_model, test_loader)
  test_accuracy_lst.append(test_accuracy)
  test2_accuracy = eval(vgg_model, test2_loader)
  test2_accuracy_lst.append(test2_accuracy)
  
  print(epoch+1, "loss :", train_loss)
  print("test_Accuracy :", test_accuracy, "test2_Accuracy :", test2_accuracy)
  
summary(vgg_model, input_size = (3,32,32))

1 loss : 2.254498469829559
test_Accuracy : 0.5 test2_Accuracy : 0.5
2 loss : 2.1581859827041625
test_Accuracy : 0.5 test2_Accuracy : 0.5
3 loss : 2.0663307309150696
test_Accuracy : 0.5 test2_Accuracy : 0.5
4 loss : 1.9772246122360229
test_Accuracy : 0.5 test2_Accuracy : 0.5
5 loss : 1.8893856883049012
test_Accuracy : 0.5 test2_Accuracy : 0.5
6 loss : 1.8014653623104095
test_Accuracy : 0.5 test2_Accuracy : 0.5
7 loss : 1.7119963824748994
test_Accuracy : 0.5 test2_Accuracy : 0.5
8 loss : 1.6198870420455933
test_Accuracy : 0.5 test2_Accuracy : 0.5
9 loss : 1.523458904027939
test_Accuracy : 0.5 test2_Accuracy : 0.5
10 loss : 1.421715748310089
test_Accuracy : 0.5 test2_Accuracy : 0.5
11 loss : 1.31413494348526
test_Accuracy : 0.5 test2_Accuracy : 0.5
12 loss : 1.2015278697013856
test_Accuracy : 0.5 test2_Accuracy : 0.5
13 loss : 1.088060975074768
test_Accuracy : 0.5 test2_Accuracy : 0.5
14 loss : 0.982010406255722
test_Accuracy : 0.5 test2_Accuracy : 0.5
15 loss : 0.8932511955499649
test_Ac

In [0]:
resnet_model = SimpleResNet().cuda()
train_loss_lst = []
test_accuracy_lst = []
test2_accuracy_lst = []
for epoch in range(total_epoch):
  train_loss = train(resnet_model, train_loader)
  train_loss_lst.append(train_loss)
  test_accuracy = eval(resnet_model, test_loader)
  test_accuracy_lst.append(test_accuracy)
  test2_accuracy = eval(resnet_model, test2_loader)
  test2_accuracy_lst.append(test2_accuracy)  
  
  print(epoch+1, "loss :", train_loss)
  print("test_Accuracy :", test_accuracy, "test2_Accuracy :", test2_accuracy)
  
summary(resnet_model, input_size = (3,32,32))

1 loss : 2.213685357570648
test_Accuracy : 0.5 test2_Accuracy : 0.5
2 loss : 2.078955900669098
test_Accuracy : 0.5 test2_Accuracy : 0.5
3 loss : 1.8134447574615478
test_Accuracy : 0.5 test2_Accuracy : 0.5
4 loss : 0.9323637038469315
test_Accuracy : 0.5 test2_Accuracy : 0.5
5 loss : 0.7082646042108536
test_Accuracy : 0.5 test2_Accuracy : 0.5
6 loss : 0.702398681640625
test_Accuracy : 0.5 test2_Accuracy : 0.5
7 loss : 0.6997823923826217
test_Accuracy : 0.5 test2_Accuracy : 0.5
8 loss : 0.6994103342294693
test_Accuracy : 0.5 test2_Accuracy : 0.5
9 loss : 0.6987123161554336
test_Accuracy : 0.5 test2_Accuracy : 0.51
10 loss : 0.6993323922157287
test_Accuracy : 0.5 test2_Accuracy : 0.5
11 loss : 0.693648836016655
test_Accuracy : 0.5 test2_Accuracy : 0.5
12 loss : 0.6965001255273819
test_Accuracy : 0.53 test2_Accuracy : 0.49
13 loss : 0.6876101523637772
test_Accuracy : 0.5 test2_Accuracy : 0.5
14 loss : 0.6922943860292434
test_Accuracy : 0.53 test2_Accuracy : 0.5
15 loss : 0.7033074736595154
